## OCSP usage

### Import libraries

In [1]:
import os
import pandas as pd 
import numpy as np
from lxml import etree 
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 200
import os
import glob
import time
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import matplotlib.patches as mpatches
import shutil
from IPython.display import clear_output
pd.set_option('display.max_rows', 500)
from manuf import manuf
from user_agents import parse
import dataframe_image as dfi
import tldextract

### Readin dataset

In [3]:
dev = pd.read_csv("../datasets/2014dev_with_dist2lib.csv")

In [4]:
ocsp = dev[['device_id', 'device_vendor', 'extension_types']]

In [5]:
ocsp['extension_types'] = ocsp['extension_types'].fillna("missing")
ocsp['extension'] = ocsp['extension_types'].apply(lambda x: x.split("+"))

<ipython-input-5-73126371e930>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ocsp['extension_types'] = ocsp['extension_types'].fillna("missing")
<ipython-input-5-73126371e930>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ocsp['extension'] = ocsp['extension_types'].apply(lambda x: x.split("+"))


### Analysis on OCSP usage

In [8]:
ocsp = ocsp.explode('extension')

In [9]:
ocsp.sample(1)

,device_id,device_vendor,extension_types,extension
1745,s44ecd8086e,google,23+65281+10+11+35+16+5+13+18+51+45+43+27+21,13


In [10]:
ocsp = ocsp.drop_duplicates(['device_id', 'device_vendor', 'extension'])[['device_id', 'device_vendor', 'extension']]

In [11]:
ocsp.shape

(17528, 3)

In [12]:
ocsp.sample(2)

,device_id,device_vendor,extension
104,s0344aa63f9,google,23
464,s142e22e7a8,amazon,65281


In [13]:
ocsp['device_vendor'].value_counts().shape

(66,)

In [14]:
withocsp = ocsp.loc[ocsp['extension'] == '5']

In [15]:
withocsp['device_id'].value_counts().shape

(648,)

In [16]:
withocsp['device_vendor'].value_counts().shape

(34,)

In [17]:
ocsp_cnt = withocsp.groupby('device_vendor')['device_id'].agg(lambda x: x.nunique()).reset_index(name='count')

In [18]:
dev_total = dev.groupby('device_vendor')['device_id'].agg(lambda x: x.nunique()).reset_index(name='total')

In [19]:
df_merge = pd.merge(dev_total, ocsp_cnt, on='device_vendor', how='inner')

In [20]:
df_merge['perc'] = df_merge['count'] / df_merge['total']

In [21]:
df_merge.sort_values(by='perc', ascending=False)

,device_vendor,total,count,perc
17,pioneer,4,4,1.000000
13,nvidia,22,22,1.000000
2,cisco,1,1,1.000000
3,fing,6,6,1.000000
28,unknown,1,1,1.000000
27,ubiquity,1,1,1.000000
7,idevices,1,1,1.000000
24,tesla,5,5,1.000000
1,amcrest,1,1,1.000000
14,onkyo,7,7,1.000000
